## fastq2vcf
### Tutorial intended to cover the analysis of human next generation sequencing (NGS) data. In particular the processing of fastq files in order to get variants in VCF format. 


#### Fastq Downloading

We are going to use public data from the Simons Genomes Diversity Project ([SGDP](https://reichdata.hms.harvard.edu/pub/datasets/sgdp/)). Specifically 7 selected genomes from each 'Region' included in the dataset. The SGDP Project ID in ENA repository is [PRJEB9586](https://www.ebi.ac.uk/ena/browser/view/PRJEB9586).

The code below fetches partial fastq files (2 million paired-end reads per individual) coming from genomes of the 7 different regions defined in the SGDP (CentralAsiaSiberia, Africa, EastAsia, WestEurasia, America, SouthAsia and Oceania). The reduction in reads per sample is to speed up the analysis but the code can be easily modified to get and work on complete genomes.

[Here](https://en.wikipedia.org/wiki/FASTQ_format) and [here](https://support.illumina.com/bulletins/2016/04/fastq-files-explained.html) links to the description and explanation of the FASTQ format.


In [1]:
import pandas as pd
import os

SGDP_ENA_PID = 'PRJEB9586'
ENA_URL = 'https://www.ebi.ac.uk/ena/data/warehouse/filereport?accession=' + SGDP_ENA_PID + '&result=read_run'
metadata_URL = 'https://sharehost.hms.harvard.edu/genetics/reich_lab/sgdp/SGDP_metadata.279public.21signedLetter.samples.txt'

download_table = pd.read_csv(ENA_URL, sep='\t')
download_table = download_table[download_table['submitted_format']=='FASTQ;FASTQ']

metadata = pd.read_csv(metadata_URL, encoding="ISO-8859-1", sep='\t')
metadata = metadata[(metadata['#Sequencing_Panel']=='C') & (metadata['Embargo']=='FullyPublic')]
metadata = metadata.drop_duplicates(subset=['Region'], keep='last')
#print(metadata.head())

download_table = download_table[download_table['library_name'].isin([x for x in metadata['Illumina_ID']])]
#print(download_table.head())

# [wd] = !pwd
for sample_ID in metadata['Illumina_ID']:
    !mkdir -p fastq/{sample_ID}
    
    print("Dowloading fastq partial files for sample:", sample_ID)
    [fastq_url] = download_table[download_table['library_name']==sample_ID]['fastq_ftp'].str.split(';').values
    [run_accession] = download_table[download_table['library_name']==sample_ID]['run_accession'].values
        
    if os.path.exists('fastq/' + sample_ID + '/' + run_accession + '_1.fastq.gz'):
        print("__fastq 1 file already present__")
    else:
        !wget --quiet -O - '{fastq_url[0]}' | zcat 2>/dev/null | head -n 8000000 - | gzip - > fastq/{sample_ID}/{run_accession}_1.fastq.gz
    if os.path.exists('fastq/' + sample_ID + '/' + run_accession + '_2.fastq.gz'):
        print("__fastq 2 file already present__")
    else:
        !wget --quiet -O - '{fastq_url[1]}' | zcat 2>/dev/null | head -n 8000000 - | gzip - > fastq/{sample_ID}/{run_accession}_2.fastq.gz


Dowloading fastq partial files for sample: LP6005443-DNA_B04
__fastq 1 file already present__
__fastq 2 file already present__
Dowloading fastq partial files for sample: LP6005441-DNA_A08
__fastq 1 file already present__
__fastq 2 file already present__
Dowloading fastq partial files for sample: LP6005441-DNA_B09
__fastq 1 file already present__
__fastq 2 file already present__
Dowloading fastq partial files for sample: LP6005441-DNA_G09
__fastq 1 file already present__
__fastq 2 file already present__
Dowloading fastq partial files for sample: LP6005519-DNA_G02
__fastq 1 file already present__
__fastq 2 file already present__
Dowloading fastq partial files for sample: LP6005442-DNA_B12
__fastq 1 file already present__
__fastq 2 file already present__
Dowloading fastq partial files for sample: LP6005443-DNA_F08
__fastq 1 file already present__
__fastq 2 file already present__


#### MD5 checksum

[MD5](https://en.wikipedia.org/wiki/MD5) is a hash function that produces a 32-character string ([checksum](https://en.wikipedia.org/wiki/Checksum)) for a given file and is widely used to check for data corruption in data files after their transmission or storage. For instance, since the checksum is unique (with very high probability) for every file, if this is the same before and after the downloading of a data file, this means the two files are the same and no data corruption occured during the transfer.

When working with relatively big datasets (hundreds-to-thousands of samples, but also smaller numbers), its not rare that some of the downloaded files experienced some sort of small data corruption even after a 'succesfull' exit status and finishing during a wget retrieval (as used above). it is a bit annoying to find out about this issue later on in other stages of the data processing, so it is always much better if one has a quick way to do a checksum test and re-download those data files not passing it. 

In this case and since we modified the original fastq files from ENA repository by reducing the number of reads in each file, we can no longer use the MD5 checksums pre-computed and provided originally for the complete files. However I have computed them in advanced for these reduced fastq files and here below there is an example on how the test can be done. 

In [2]:
md5_checksums = pd.read_csv('md5_checksums', sep='\t')

for sample_ID in metadata['Illumina_ID']:
    
    print("Checksum test for sample:", sample_ID)
    [run_accession] = download_table[download_table['library_name']==sample_ID]['run_accession'].values
    
    [checksum_1] = !md5sum fastq/{sample_ID}/{run_accession}_1.fastq.gz | cut -d' ' -f1
    [expected_checksum_1] = md5_checksums[md5_checksums['Sample_ID']==sample_ID]['fastq_1'].values
    [checksum_2] = !md5sum fastq/{sample_ID}/{run_accession}_2.fastq.gz | cut -d' ' -f1
    [expected_checksum_2] = md5_checksums[md5_checksums['Sample_ID']==sample_ID]['fastq_2'].values
    
    if checksum_1 == expected_checksum_1:
        print("[ fastq file 1 ] Succesfull test")
    else:
        print("[ fastq file 1 ] checksum test unsuccesfull...\nRe-dowloading fastq file 1", )
        !wget --quiet -O - '{fastq_url[0]}' | zcat 2>/dev/null | head -n 8000000 - | gzip - > fastq/{sample_ID}/{run_accession}_1.fastq.gz
    
    if checksum_2 == expected_checksum_2:
        print("[ fastq file 2 ] Succesfull test")
    else:
        print("[ fastq file 2 ] checksum test unsuccesfull...\nRe-dowloading fastq file 1", )
        !wget --quiet -O - '{fastq_url[1]}' | zcat 2>/dev/null | head -n 8000000 - | gzip - > fastq/{sample_ID}/{run_accession}_1.fastq.gz


Checksum test for sample: LP6005443-DNA_B04
[ fastq file 1 ] Succesfull test
[ fastq file 2 ] Succesfull test
Checksum test for sample: LP6005441-DNA_A08
[ fastq file 1 ] Succesfull test
[ fastq file 2 ] Succesfull test
Checksum test for sample: LP6005441-DNA_B09
[ fastq file 1 ] Succesfull test
[ fastq file 2 ] Succesfull test
Checksum test for sample: LP6005441-DNA_G09
[ fastq file 1 ] Succesfull test
[ fastq file 2 ] Succesfull test
Checksum test for sample: LP6005519-DNA_G02
[ fastq file 1 ] Succesfull test
[ fastq file 2 ] Succesfull test
Checksum test for sample: LP6005442-DNA_B12
[ fastq file 1 ] Succesfull test
[ fastq file 2 ] Succesfull test
Checksum test for sample: LP6005443-DNA_F08
[ fastq file 1 ] Succesfull test
[ fastq file 2 ] Succesfull test


#### Fastq Quality Check

It is generally important to run a quality check on the raw fastq files before further processing. [FASTQC](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/) is a program that allows to do this in a simple way. On a single run it provides an overview of various statistics on the data (quality per base and per read, distribution of read lengths, etc) and generates an .html containing graphics that can be opened in a web browser for visualization. In the webpage of FASTQ it is possible to find [detailed explanations](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/Help/) of each quality report (aka 'modular analysis') as well as examples of [bad](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/bad_sequence_fastqc.html) and [good](https://www.bioinformatics.babraham.ac.uk/projects/fastqc/good_sequence_short_fastqc.html) data. Here below an example of how one can run this program and he output generated for one of the fastq files used here.

Other fastq quality control tools include:  

[FASTX-Toolkit](http://hannonlab.cshl.edu/fastx_toolkit/)  
[PRINSEQ](https://edwards.sdsu.edu/cgi-bin/prinseq/prinseq.cgi)

In [3]:
for sample_ID in metadata['Illumina_ID']:
    
    print("Fastq file quality check for sample:", sample_ID)
    [run_accession] = download_table[download_table['library_name']==sample_ID]['run_accession'].values
    
    if os.path.exists('fastq/' + sample_ID + '/' + run_accession + '_1_fastqc.zip'):
        print("__quality check for fastq file 1 already present__")
    else:
        !fastqc fastq/{sample_ID}/{run_accession}_1.fastq.gz

    if os.path.exists('fastq/' + sample_ID + '/' + run_accession + '_2_fastqc.zip'):
        print("__quality check for fastq file 2 already present__")
    else:
        !fastqc fastq/{sample_ID}/{run_accession}_2.fastq.gz
    
from IPython.display import IFrame
src='https://htmlpreview.github.io/?https://github.com/JRodrigoF/fastq2VCF/blob/master/fastq/LP6005441-DNA_A08/ERR1419093_1_fastqc.html'
IFrame(src, width=1000, height=600)

Fastq file quality check for sample: LP6005443-DNA_B04
__quality check for fastq file 1 already present__
__quality check for fastq file 2 already present__
Fastq file quality check for sample: LP6005441-DNA_A08
__quality check for fastq file 1 already present__
__quality check for fastq file 2 already present__
Fastq file quality check for sample: LP6005441-DNA_B09
__quality check for fastq file 1 already present__
__quality check for fastq file 2 already present__
Fastq file quality check for sample: LP6005441-DNA_G09
__quality check for fastq file 1 already present__
__quality check for fastq file 2 already present__
Fastq file quality check for sample: LP6005519-DNA_G02
__quality check for fastq file 1 already present__
__quality check for fastq file 2 already present__
Fastq file quality check for sample: LP6005442-DNA_B12
__quality check for fastq file 1 already present__
__quality check for fastq file 2 already present__
Fastq file quality check for sample: LP6005443-DNA_F08
__q

#### Adapter Removal

Library preparation for Illumina short-read sequencing involves the addition of adapter sequences to the ends of the sequence fragments in order to allow them to attach to the surface of the flowcell and undergo amplification rounds followed by [sequencing by synthesis](https://www.youtube.com/watch?v=fCd6B5HRaZ8). Whenever the read length is greater than a particular fragment insert size, the sequenced read will contain a portion of the adapter sequence at its [3' end](https://support.illumina.com/bulletins/2016/04/adapter-trimming-why-are-adapter-sequences-trimmed-from-only-the--ends-of-reads.html). Fastq [Illumina generation pipelines](https://emea.support.illumina.com/content/dam/illumina-support/documents/documentation/software_documentation/bcl2fastq/bcl2fastq2-v2-20-software-guide-15051736-03.pdf) have an option to automatically remove these adapter sequences, nevertheless its recommended to check for their presence using tools like FASTQC as done in the previous section (see "Adapter Content" analysis). 

As we observed in the previous section, - using partial fastq files - SGDP data appear to be free form adapter sequences. However, as stated in the [original publication](https://static-content.springer.com/esm/art%3A10.1038%2Fnature18964/MediaObjects/41586_2016_BFnature18964_MOESM204_ESM.pdf), these data were found to still contain small percentages of adapters and further trimmed to remove them. Checking and ensuring that final fastq files do not contain high proportions of adapter sequences becomes even more important in some library preparations and sequencing experiments such as RNA-seq and [ancient dna](https://academic.oup.com/nar/article/42/18/e141/2434537). 

#### Mapping 


Additional Relevant Links: 

[https://jupyter.org/](https://jupyter.org/)  
[https://pandas.pydata.org/](https://pandas.pydata.org/)
